# Training module

In [ ]:
#import library
import csv
import pandas as pd
import pickle 
import numpy as np
import cv2
import mediapipe as mp
import time
import shutil
import os
import sys

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score 
import joblib

In [ ]:
# Grabbing the Holistic Model from Mediapipe and
mp_holistic = mp.solutions.holistic

# Initializing the Model
holistic_model = mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
 
# Initializing the drawing utils for drawing the facial landmarks on image
mp_drawing = mp.solutions.drawing_utils

# Test Camera

In [ ]:
def Test_cam():
    # (0) in VideoCapture is used to connect to your computer's default camera
    capture = cv2.VideoCapture(0)

    # Initializing current time and precious time for calculating the FPS
    previousTime = 0
    currentTime = 0

    start_time = time.time()

    while capture.isOpened():
        # capture frame by frame
        ret, frame = capture.read()

        elapsed_time = time.time() - start_time

        # resizing the frame for better view
        frame = cv2.resize(frame, (860,645))

        # Converting the from BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Making predictions using holistic model
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = holistic_model.process(image)
        image.flags.writeable = True

        # Converting back the RGB image to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(
            image, 
            results.pose_landmarks, 
            mp_holistic.POSE_CONNECTIONS,            
        )
        # Calculating the FPS
        currentTime = time.time()
        fps = 1 / (currentTime-previousTime)
        previousTime = currentTime

        # represents the top left corner of rectangle
        start_point = (200, 10)

        # represents the bottom right corner of rectangle
        end_point = (680, 630)

        # Blue color in BGR
        color = (0,255,255)

        # Line thickness of 2 px
        thickness = 4

        # Using cv2.rectangle() method
        # Draw a rectangle with blue line borders of thickness of 2 px
        cv2.rectangle(image, start_point, end_point, color, thickness)


        # Displaying FPS on the image
        cv2.putText(image, str(int(fps))+" FPS", (10, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)

        # Display Timer
        cv2.putText(image, 'TIME'
                    , (300,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, f"{int(elapsed_time)} Sec"
                    , (350,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Display the resulting image
        cv2.imshow("Test Camera", image)

        # Enter key 'q' to break the loop
        if cv2.waitKey(5) & 0xFF == ord('q') or cv2.waitKey(5) & int(elapsed_time) == 5:
            break

    # When all the process is done
    # Release the capture and destroy all windows
    capture.release()
    cv2.destroyAllWindows()
    
#Test_cam()

# Data acquisition

In [ ]:
waistFeetAndLegRaises = ["wflr0","wflr1","wflr2","wflr3"]
stompingAndBent = ["sab0","sab1","sab2"]
fistAndStride = ["fas0","fas1","fas2","fas3"]
stretchOutAndStepBack  = ["ss0","ss1","ss2","ss3"]

In [ ]:
model_name = "stretchOutAndStepBack"

#create folder of the model 
modelFolerPath = f"Dataset/{model_name}"
if not os.path.exists(modelFolerPath):
    os.makedirs(modelFolerPath)

In [ ]:
class_name = stretchOutAndStepBack[0]

# Absolute path of a file
go_src_path = r"Dataset/predataset.csv"
go_dst_path = f"Dataset/{model_name}/predataset.csv"
data_path = f"Dataset/{model_name}/{class_name}.csv"

# Copying and rename the file
if not os.path.isfile(data_path):
    shutil.copy(go_src_path, go_dst_path)
    os.rename(go_dst_path, data_path)

In [ ]:
# Test Camera
Test_cam()

In [ ]:
start_time = time.time()

# (0) in VideoCapture is used to connect to your computer's default camera
capture = cv2.VideoCapture(0)
 
while capture.isOpened():
    # capture frame by frame
    ret, frame = capture.read()
    
    elapsed_time = time.time() - start_time
 
    # resizing the frame for better view
    frame = cv2.resize(frame, (860,645))
 
    # Converting the from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
 
    # Making predictions using holistic model
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = holistic_model.process(image)
    image.flags.writeable = True
 
    # Converting back the RGB image to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    mp_drawing.draw_landmarks(
        image, 
        results.pose_landmarks, 
        mp_holistic.POSE_CONNECTIONS,            
    )
    
    try:
        pose = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())

        pose_row.insert(0, class_name)
            
        with open(data_path, mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',')
            csv_writer.writerow(pose_row)
        
        # represents the top left corner of rectangle
        start_point = (200, 10)
        
        # represents the bottom right corner of rectangle
        end_point = (680, 630)
        
        # Blue color in BGR
        color = (0,255,255)
        
        # Line thickness of 2 px
        thickness = 4
        
        # Using cv2.rectangle() method
        # Draw a rectangle with blue line borders of thickness of 2 px
        cv2.rectangle(image, start_point, end_point, color, thickness)

        # Get status box
        cv2.rectangle(image, (0,0), (500, 60), (255, 255, 255), -1)
        
        # Display Timer
        cv2.putText(image, 'TIME'
                    , (300,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, f"{int(elapsed_time)} Sec"
                    , (350,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            
    except:
        cv2.putText(image, str('Out of Frame')
                    , (210,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    

    # Display the resulting image
    cv2.imshow("Test Data", image)
 
    # Enter key 'q' to break the loop
    if cv2.waitKey(5) & 0xFF == ord('q') or cv2.waitKey(5) & int(elapsed_time) == 20:
        break
 
# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()

# TRAIN MODEL

In [ ]:
# model_name = "stretchOutAndStepBack"
modelFolerPath = f"Dataset/{model_name}"

In [ ]:
# iterate over files in data directory
list_class = []
list_data = []
directory = modelFolerPath+"/"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        list_data.append(pd.read_csv(f))
        x = filename.split(".")
        list_class.append(x[0])

In [ ]:
sys.stdout.write("[  There is " + str(len(list_class)) + " class for this model  ]\n")
print("==============================================================================")
for i in list_class:
    print(i)

print("==============================================================================")
df_model = pd.concat(list_data,ignore_index=True)
print("df_model is now complete")

In [ ]:
df_model = df_model.drop(0)

In [ ]:
df_model 

In [ ]:
x = df_model .drop('class', axis=1) 
y = df_model ['class']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=999)

In [ ]:
pipelines = {
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier(), verbose = 1)
}

In [ ]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

In [ ]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test, yhat))

In [ ]:
save_dst= f"Model/{mode_name}.pkl"
pickle.dump(fit_models['rf'],open(save_dst, 'wb'))